In [1]:
import pandas as pd
import psycopg2


import server.season.s2019.view.rankingtable as rt
import server.model.event as sme
import server.model.connection as smc

In [2]:

sme.EventDal.set_current_event('wayak', '2019')
sme.EventDal.set_current_match('020-q')

'current match 020-q'

In [3]:
df = rt.ranking_df()
df.head()


,centerCargo,centerHatch,climb,closeCargo,closeHatch,collision,crossCenterLine,crossHabLine,csCargo,csHatch,...,climb1Points,climb2Points,climb3Points,avgClimbPoints,habPoints,avgHabPoints,avgPoints,level1hab,dontMove,Defense
1318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1.0,...,3.0,6.0,12.0,5.25,9.0,2.25,18.25,3.0,1.0,0.00
1595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,...,9.0,0.0,0.0,2.25,6.0,1.50,10.75,0.0,3.0,0.00
2148,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,...,9.0,0.0,0.0,2.25,6.0,1.50,11.75,2.0,2.0,0.00
2522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,5.0,0.0,...,3.0,18.0,0.0,5.25,18.0,4.50,20.00,0.0,1.0,0.75
2605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,18.0,0.0,4.50,9.0,2.25,6.75,3.0,1.0,1.25


In [4]:
newdf = df[['avgCargo', 'avgHatch']].copy()
newdf

,avgCargo,avgHatch
1318,1.25,3.50
1595,1.00,2.00
2148,0.00,4.00
2522,2.25,1.75
2605,0.00,0.00
2811,2.25,0.00
2910,3.75,6.50
2926,0.00,0.25
2944,0.00,0.25
3218,1.75,1.50


In [5]:
stanDev = newdf.std(axis = 0)
stanDev

avgCargo    1.664876
avgHatch    1.634981
dtype: float64

In [6]:
cstd = stanDev.at['avgCargo']
cstd


1.6648759196880074

In [7]:
hstd = stanDev.at['avgHatch']
hstd


1.6349805168021363

In [8]:
def getSim(team1, team2, newdf):
    newdf = newdf.T
    tdf = newdf[[team1, team2]].copy()
    listy = tdf.values.tolist()
    sim = ((((listy[0][0] - listy[0][1]) ** 2) / cstd) + (((listy[1][0] - listy[1][1]) ** 2) / hstd)) ** 0.5
    return sim

In [9]:
teamslist = newdf.index.tolist()
len(teamslist)

31

In [10]:
simData = []
for i in range(0, len(teamslist), 1):
    for j in range(i + 1, len(teamslist), 1):
        sim = getSim(teamslist[i], teamslist[j], newdf)
        simData.append([teamslist[i], teamslist[j], sim])
simData

[['1318', '1595', 1.1889926505025712],
 ['1318', '2148', 1.044708284399623],
 ['1318', '2522', 1.5728179703776661],
 ['1318', '2605', 2.9036102357435984],
 ['1318', '2811', 2.8448355637557694],
 ['1318', '2910', 3.0428089433483456],
 ['1318', '2926', 2.7200790448554857],
 ['1318', '2944', 2.7200790448554857],
 ['1318', '3218', 1.61141976978505],
 ['1318', '3220', 2.5490904194988637],
 ['1318', '3693', 2.9036102357435984],
 ['1318', '3711', 3.1587487779796293],
 ['1318', '3712', 2.9036102357435984],
 ['1318', '4061', 2.6405344260133368],
 ['1318', '4104', 2.9036102357435984],
 ['1318', '4125', 0.9744183392023533],
 ['1318', '4131', 4.611495352134723],
 ['1318', '4692', 2.92325501760706],
 ['1318', '4911', 1.044708284399623],
 ['1318', '4980', 2.039739821149601],
 ['1318', '5295', 1.839842578184344],
 ['1318', '5468', 1.044708284399623],
 ['1318', '5803', 2.89224066671988],
 ['1318', '5920', 2.607332860979058],
 ['1318', '6076', 2.9036102357435984],
 ['1318', '6442', 2.7593612863050123],

In [11]:
simDf = pd.DataFrame(simData, columns = ['Team 1', 'Team 2', 'Similarity']) 
simDf[(simDf['Team 1'] == '2910') | (simDf['Team 2'] == '2910')].sort_values('Similarity')

,Team 1,Team 2,Similarity
5,1318,2910,3.042809
62,2148,2910,3.502749
180,2910,5468,3.502749
177,2910,4911,3.502749
89,2522,2910,3.892468
174,2910,4125,4.016263
34,1595,2910,4.114347
167,2910,3218,4.206338
181,2910,5803,4.305728
176,2910,4692,4.696398
